# Деревья

Ключевые алгоритмы:

- ID3: использует энтропию и прирост информации
- C4.5: улучшение ID3, может работать с непрерывными признаками
- CART: бинарное дерево, использует индекс Джини для классификации

Критерии разбиения:

- Энтропия: $H(S) = -\sum_{i=1}^{c} p_i \log_2(p_i)$
- Индекс Джини: $G(S) = 1 - \sum_{i=1}^{c} p_i^2$

## Руками

In [ ]:
import pandas as pd
import numpy as np

data = {
    'Погода': ['Солнечно', 'Солнечно', 'Пасмурно', 'Дождь', 'Дождь', 'Дождь', 'Пасмурно', 'Солнечно', 'Солнечно', 'Дождь', 'Солнечно', 'Пасмурно', 'Пасмурно', 'Дождь'],
    'Температура': ['Жарко', 'Жарко', 'Жарко', 'Умеренно', 'Прохладно', 'Прохладно', 'Прохладно', 'Умеренно', 'Прохладно', 'Умеренно', 'Умеренно', 'Умеренно', 'Жарко', 'Умеренно'],
    'Влажность': ['Высокая', 'Высокая', 'Высокая', 'Высокая', 'Нормальная', 'Нормальная', 'Нормальная', 'Высокая', 'Нормальная', 'Нормальная', 'Нормальная', 'Высокая', 'Нормальная', 'Высокая'],
    'Ветер': ['Нет', 'Да', 'Нет', 'Нет', 'Нет', 'Да', 'Да', 'Нет', 'Нет', 'Нет', 'Да', 'Да', 'Нет', 'Да'],
    'Играть в теннис': ['Нет', 'Нет', 'Да', 'Да', 'Да', 'Нет', 'Да', 'Нет', 'Да', 'Да', 'Да', 'Да', 'Да', 'Нет']
}

df = pd.DataFrame(data)
df

In [ ]:
# Функция для расчета энтропии
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    entropy = 0
    for i in range(len(elements)):
        prob = counts[i] / sum(counts)
        entropy += -prob * np.log2(prob)
    return entropy

# Функция для расчета прироста информации
def information_gain(data, split_attribute, target_attribute="Играть в теннис"):
    # Энтропия всего набора данных
    total_entropy = entropy(data[target_attribute])
    
    # Значения выбранного атрибута
    vals, counts = np.unique(data[split_attribute], return_counts=True)
    
    # Взвешенная сумма энтропии после разбиения
    weighted_entropy = 0
    for i in range(len(vals)):
        subset_entropy = entropy(data.where(data[split_attribute] == vals[i]).dropna()[target_attribute])
        weighted_entropy += (counts[i]/sum(counts)) * subset_entropy
        
    # Прирост информации
    information_gain = total_entropy - weighted_entropy
    return information_gain

# Расчет прироста информации для каждого атрибута
for attribute in ['Погода', 'Температура', 'Влажность', 'Ветер']:
    print(f"Прирост информации для {attribute}: {information_gain(df, attribute):.4f}")

# Лучший атрибут для первого разбиения - тот, для которого прирост информации максимален


## Sklearn'ом

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names
class_names = iris.target_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(X_train, y_train)

train_accuracy = clf.score(X_train, y_train)
test_accuracy = clf.score(X_test, y_test)
print(f"Точность на обучающей выборке: {train_accuracy:.4f}")
print(f"Точность на тестовой выборке: {test_accuracy:.4f}")

plt.figure(figsize=(20, 10))
plot_tree(clf, filled=True, feature_names=feature_names, class_names=class_names, rounded=True)
plt.title("Дерево решений для классификации ирисов")
plt.show()

importance = clf.feature_importances_
for i, feature in enumerate(feature_names):
    print(f"Важность признака '{feature}': {importance[i]:.4f}")

## Гиперпараметры деревьев

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
import numpy as np

max_depths = np.arange(1, 20)
train_scores = []
test_scores = []

for depth in max_depths:
    clf = DecisionTreeClassifier(max_depth=depth, random_state=42)
    clf.fit(X_train, y_train)
    train_scores.append(clf.score(X_train, y_train))
    test_scores.append(clf.score(X_test, y_test))

plt.figure(figsize=(10, 6))
plt.plot(max_depths, train_scores, 'o-', label='Обучающая выборка')
plt.plot(max_depths, test_scores, 'o-', label='Тестовая выборка')
plt.xlabel('Максимальная глубина дерева')
plt.ylabel('Точность')
plt.title('Влияние глубины дерева на точность')
plt.legend()
plt.grid(True)
plt.show()

param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), 
                          param_grid=param_grid, 
                          cv=5, 
                          scoring='accuracy', 
                          return_train_score=True)
grid_search.fit(X, y)

print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучшая точность: {grid_search.best_score_:.4f}")

results = grid_search.cv_results_
for depth in [3, 5, 7]:
    mask = results['param_max_depth'] == depth
    plt.figure(figsize=(10, 6))
    
    for split in [2, 5, 10]:
        split_mask = mask & (results['param_min_samples_split'] == split)
        plt.plot(
            results['param_min_samples_leaf'][split_mask],
            results['mean_test_score'][split_mask],
            'o-', label=f'min_samples_split={split}'
        )
    
    plt.xlabel('min_samples_leaf')
    plt.ylabel('Score')
    plt.title(f'Точность при max_depth={depth}')
    plt.legend()
    plt.grid(True)
    plt.show()

## Isolation tree

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np

n_samples = 300
n_outliers = 15
X, _ = make_blobs(n_samples=n_samples-n_outliers, centers=[[0, 0], [3, 3]], cluster_std=0.5, random_state=42)

rng = np.random.RandomState(42)
X_outliers = rng.uniform(low=-4, high=8, size=(n_outliers, 2))
X = np.vstack([X, X_outliers])

clf = IsolationForest(contamination=n_outliers/n_samples, random_state=42)
y_pred = clf.fit_predict(X)
scores = clf.decision_function(X)

plt.figure(figsize=(10, 7))
plt.scatter(X[:, 0], X[:, 1], c=y_pred, cmap='coolwarm')
plt.colorbar(label='Выбросы (-1) vs. Нормальные точки (1)')
plt.title('Обнаружение выбросов с помощью Isolation Forest')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

n_transactions = 10000
n_frauds = 100

normal_transactions = np.random.randn(n_transactions - n_frauds, 2)
normal_transactions[:, 0] = np.abs(normal_transactions[:, 0]) * 100
normal_transactions[:, 1] = np.abs(normal_transactions[:, 1]) * 10

fraud_transactions = np.random.randn(n_frauds, 2)
fraud_transactions[:, 0] = np.abs(fraud_transactions[:, 0]) * 500 + 200
fraud_transactions[:, 1] = np.abs(fraud_transactions[:, 1]) * 3 + 20

X = np.vstack([normal_transactions, fraud_transactions])
true_labels = np.zeros(n_transactions)
true_labels[n_transactions - n_frauds:] = 1

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

clf = IsolationForest(contamination=n_frauds/n_transactions, random_state=42)
y_pred = clf.fit_predict(X_scaled)
y_pred = [1 if i == -1 else 0 for i in y_pred]

from sklearn.metrics import confusion_matrix, classification_report

print("Матрица ошибок:")
print(confusion_matrix(true_labels, y_pred))
print("\nОтчет о классификации:")
print(classification_report(true_labels, y_pred))

plt.figure(figsize=(12, 8))
plt.scatter(X[:, 0], X[:, 1], c=y_pred, cmap='coolwarm', alpha=0.7)
plt.xlabel('Сумма транзакции')
plt.ylabel('Время суток')
plt.title('Обнаружение мошеннических транзакций с помощью Isolation Forest')
plt.colorbar(label='Предсказание: Мошенничество (1) vs. Нормальная транзакция (0)')
plt.grid(True)
plt.show()

## Инкрементальные деревья

In [ ]:
%pip install river

In [ ]:
from river import tree
from river import metrics
from river import datasets
from river import evaluate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def generate_drift_stream(n_samples=10000, n_features=2, n_classes=2, random_state=42):
    rng = np.random.RandomState(random_state)
    
    X = np.zeros((n_samples, n_features))
    y = np.zeros(n_samples)
    
    for i in range(n_samples):
        if i < n_samples / 3:
            center = 0
        elif i < 2 * n_samples / 3:
            center = 2
        else:
            center = 4
            
        if rng.rand() > 0.5:
            X[i, 0] = rng.normal(center, 1)
            X[i, 1] = rng.normal(0, 1)
            y[i] = 0
        else:
            X[i, 0] = rng.normal(0, 1)
            X[i, 1] = rng.normal(center, 1)
            y[i] = 1
    
    stream = []
    for i in range(n_samples):
        stream.append(({'x0': X[i, 0], 'x1': X[i, 1]}, y[i]))
    
    return stream

stream = generate_drift_stream()

model = tree.HoeffdingTreeClassifier(
    grace_period=100,
    split_criterion='entropy',
    max_depth=5
)

metric = metrics.Accuracy()

results = []
for i, (x, y) in enumerate(stream):
    # Сначала предсказываем
    y_pred = model.predict_one(x)
    
    # Затем обновляем метрику
    if i > 0:  # Пропускаем первое наблюдение, так как нет предсказания
        metric.update(y, y_pred)
        results.append((i, metric.get()))
    
    # Обучаем модель
    model.learn_one(x, y)
    
    # Выводим текущую точность каждые 1000 примеров
    if i % 1000 == 0 and i > 0:
        print(f"Наблюдение {i}, Точность: {metric.get():.4f}")

results_df = pd.DataFrame(results, columns=['Наблюдение', 'Точность'])
plt.figure(figsize=(12, 6))
plt.plot(results_df['Наблюдение'], results_df['Точность'])
plt.xlabel('Количество наблюдений')
plt.ylabel('Точность')
plt.title('Изменение точности инкрементального дерева решений со временем')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

batch_size = 1000
batches = [stream[i:i+batch_size] for i in range(0, len(stream), batch_size)]

batch_results = []
for batch_idx, batch in enumerate(batches[:-1]):
    X_train = pd.DataFrame([x for x, _ in batch])
    y_train = np.array([y for _, y in batch])
    
    test_batch = batches[batch_idx + 1]
    X_test = pd.DataFrame([x for x, _ in test_batch])
    y_test = np.array([y for _, y in test_batch])
    
    batch_model = DecisionTreeClassifier(max_depth=5, random_state=42)
    batch_model.fit(X_train, y_train)
    
    accuracy = batch_model.score(X_test, y_test)
    batch_results.append(((batch_idx + 1) * batch_size, accuracy))
    
    print(f"Батч {batch_idx+1}, Точность обычного дерева: {accuracy:.4f}")

plt.figure(figsize=(12, 6))
plt.plot(results_df['Наблюдение'], results_df['Точность'], label='Инкрементальное дерево')
plt.plot(*zip(*batch_results), 'o-', label='Обычное дерево (переобучение на батчах)')
plt.xlabel('Количество наблюдений')
plt.ylabel('Точность')
plt.title('Сравнение инкрементального и обычного деревьев решений')
plt.legend()
plt.grid(True)
plt.show()

print("Изученные правила инкрементального дерева:")
print(model)

## Кредитный скоринг

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
n_samples = 1000

age = np.random.normal(35, 10, n_samples)
income = np.exp(np.random.normal(10, 0.7, n_samples)) / 1000
credit_history = np.random.choice([0, 0.5, 1], n_samples, p=[0.2, 0.3, 0.5])
employment_years = np.random.gamma(2, 3, n_samples)
debt_to_income = np.random.beta(2, 5, n_samples)

credit_approval = (0.2 * age + 0.4 * income + 0.25 * credit_history + 
                  0.1 * employment_years - 0.3 * debt_to_income + 
                  np.random.normal(0, 0.1, n_samples))
credit_approval = (credit_approval > np.median(credit_approval)).astype(int)

data = pd.DataFrame({
    'возраст': age,
    'доход': income,
    'кредитная_история': credit_history,
    'стаж_работы': employment_years,
    'соотношение_долга_к_доходу': debt_to_income,
    'одобрен_кредит': credit_approval
})

X = data.drop('одобрен_кредит', axis=1)
y = data['одобрен_кредит']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f"Точность: {accuracy_score(y_test, y_pred):.4f}")
print("\nОтчет о классификации:")
print(classification_report(y_test, y_pred))
print("\nМатрица ошибок:")
print(confusion_matrix(y_test, y_pred))

plt.figure(figsize=(20, 10))
plot_tree(clf, filled=True, feature_names=X.columns, class_names=['Отказ', 'Одобрено'], rounded=True)
plt.title("Дерево решений для кредитного скоринга")
plt.show()

importance = clf.feature_importances_
feature_importance = pd.DataFrame({
    'Признак': X.columns,
    'Важность': importance
}).sort_values('Важность', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Признак'], feature_importance['Важность'])
plt.xlabel('Важность')
plt.title('Важность признаков в модели кредитного скоринга')
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

print("Примеры правил принятия решений:")
feature_names = X.columns
tree_ = clf.tree_
node_indicator = clf.decision_path(X_test)
leaf_id = clf.apply(X_test)

for sample_id in range(5):
    print(f"\nПример {sample_id+1}:")
    node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                        node_indicator.indptr[sample_id + 1]]
    
    for node_id in node_index:
        if leaf_id[sample_id] == node_id:
            print(f"  → Решение: {'Одобрено' if clf.predict([X_test.iloc[sample_id]])[0] == 1 else 'Отказ'}")
            continue
            
        if (X_test.iloc[sample_id, tree_.feature[node_id]] <= tree_.threshold[node_id]):
            threshold_sign = "<="
        else:
            threshold_sign = ">"
            
        feature = feature_names[tree_.feature[node_id]]
        threshold = tree_.threshold[node_id]
        print(f"  {feature} {threshold_sign} {threshold:.2f}")